## Imports

In [1]:
import pandas as pd
from string import ascii_uppercase as alfabeto
import pickle
from bs4 import BeautifulSoup
import requests
pd.set_option('display.max_columns', None)

## Data extraction

### Node Definition

In [7]:
csv_file=('formula1_2020season_raceResults.csv')
def extract_data(csv: str):
    loaded_data = pd.read_csv(csv)
    return loaded_data

data = extract_data(csv_file)
race_tracks=data['Track'].unique()
drivers=data['Driver'].unique()
teams=data['Team'].unique()
data

,Track,Position,No,Driver,Team,Starting Grid,Laps,Total Time/Gap/Retirement,Points,Fastest Lap
0,Austria,1,77,Valtteri Bottas,Mercedes,1,71,1:30:55.739,25,No
1,Austria,2,16,Charles Leclerc,Ferrari,7,71,+2.700,18,No
2,Austria,3,4,Lando Norris,McLaren Renault,3,71,+5.491,16,Yes
3,Austria,4,44,Lewis Hamilton,Mercedes,5,71,+5.689,12,No
4,Austria,5,55,Carlos Sainz,McLaren Renault,8,71,+8.903,10,No
...,...,...,...,...,...,...,...,...,...,...
335,Abu Dhabi,16,99,Antonio Giovinazzi,Alfa Romeo Racing Ferrari,14,54,+1 lap,0,No
336,Abu Dhabi,17,6,Nicholas Latifi,Williams Mercedes,18,54,+1 lap,0,No
337,Abu Dhabi,18,20,Kevin Magnussen,Haas Ferrari,20,54,+1 lap,0,No
338,Abu Dhabi,19,51,Pietro Fittipaldi,Haas Ferrari,17,53,+2 laps,0,No


In [15]:
def transform_data(data):
    # Transformation based on drivers
    piloto_table = pd.DataFrame({
        'Circuit': data['Track'],
        'Starting Position': data['Starting Grid'],
        'Finish Position': data['Position'],
        'Points': data['Points'],
    })
    piloto_table['After race total points'] = piloto_table['Points'].cumsum()

    # Transformation based on teams
    team_table = pd.DataFrame({
        'Circuit': data['Track'],
        'Team': data['Team'],
        'Points': data['Points'],
    })
    team_table['After race total team points'] = team_table.groupby('Team')['Points'].cumsum()

    # Transformation based on circuits
    track_table = pd.DataFrame({
        'Circuit': data['Track'],
        'Points': data['Points'],
    })
    track_table['After race total circuit points'] = track_table.groupby('Circuit')['Points'].cumsum()

    return piloto_table, team_table, track_table


In [9]:
node_graph = {
    'extract_data_node': ['transform_data_node'],
    'transform_data_node': [],
}

In [10]:
def execute_node(node, data):
    if node == 'extract_data_node':
        return extract_data(data)
    elif node == 'transform_data_node':
        return transform_data(data)

def execute_dag(dag, data):
    node_results = {}
    for node, dependencies in dag.items():
        if not dependencies:
            # No dependencies, execute the node
            node_results[node] = execute_node(node, data)
        else:
            # Dependencies exist, execute them first
            dependency_results = [node_results[dep] for dep in dependencies]
            # Execute the current node using results of dependencies
            node_results[node] = execute_node(node, *dependency_results)

    return node_results

In [11]:
# Ejecutar el DAG
resultados_nodos = execute_dag(node_graph, csv_file)

# Imprimir los resultados de transformación
transformed_data = resultados_nodos['transform_data_node']
print("Resultados basados en pilotos:\n")
print(transformed_data[0].to_string(index=False))
print('\n' + '=' * 50 + '\n')

print("Resultados basados en equipos:\n")
print(transformed_data[1].to_string(index=False))
print('\n' + '=' * 50 + '\n')

print("Resultados basados en circuitos:\n")
print(transformed_data[2].to_string(index=False))


KeyError: 'transform_data_node'